In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
os.environ['TF_ENABLE_ONEDNN_OPTS']='0'
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from tensorflow import keras

In [2]:
ALL_DATA_DIR = '/media/windows/'

aftraining = ALL_DATA_DIR + 'AffectNet_train_set/seven/'
afvalidation = ALL_DATA_DIR + 'AffectNet_val_set/seven/'
scaled_wo_rotate = ALL_DATA_DIR + "scaled_wo_rotate/"
scaled_wo_rotate_training = scaled_wo_rotate + "training/"
scaled_wo_rotate_validation = scaled_wo_rotate + "validation/"

In [3]:
batch_size = 32 
img_height = 224
img_width = 224 

In [4]:
v1 = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_validation, labels = 'inferred', 
            image_size=(img_height,img_width), batch_size=None, seed=42)

Found 50947 files belonging to 7 classes.


In [5]:
t1 = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_training, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=None, seed=42)

Found 399839 files belonging to 7 classes.


In [6]:
v2 = tf.keras.utils.image_dataset_from_directory(afvalidation, labels = 'inferred', 
            image_size=(img_height,img_width), batch_size=None, seed=42)

Found 3500 files belonging to 7 classes.


In [7]:
t2 = tf.keras.utils.image_dataset_from_directory(aftraining, labels = 'inferred', 
            image_size=(img_height,img_width), batch_size=None, seed=42)

Found 283901 files belonging to 7 classes.


In [8]:
ds_train = t2.concatenate(t1)
ds_valid = v2.concatenate(v1)

In [9]:
validset = ds_valid.batch(batch_size).prefetch(1)
trainset = ds_train.batch(batch_size).prefetch(1)

In [10]:
base_model = keras.applications.EfficientNetB4(input_shape=(224,224,3), include_top=False, weights='imagenet')
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(7, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

In [11]:
for layer in base_model.layers:
    layer.trainable = False
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

In [12]:
model.fit(trainset, validation_data=validset, epochs=3)

Epoch 1/3
21367/21367 [==============================] - 3395s 158ms/step - loss: 1.2436 - accuracy: 0.5334 - val_loss: 1.3489 - val_accuracy: 0.4944
Epoch 2/3
21367/21367 [==============================] - 3382s 158ms/step - loss: 1.2127 - accuracy: 0.5476 - val_loss: 1.3337 - val_accuracy: 0.4921
Epoch 3/3
21367/21367 [==============================] - 3377s 158ms/step - loss: 1.2064 - accuracy: 0.5509 - val_loss: 1.3138 - val_accuracy: 0.5077


In [18]:
model1 = model

In [24]:
model = model1

In [25]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, 224, 224, 3)  0           ['normalization[0][0]']      

 block2a_expand_conv (Conv2D)   (None, 112, 112, 14  3456        ['block1b_add[0][0]']            
                                4)                                                                
                                                                                                  
 block2a_expand_bn (BatchNormal  (None, 112, 112, 14  576        ['block2a_expand_conv[0][0]']    
 ization)                       4)                                                                
                                                                                                  
 block2a_expand_activation (Act  (None, 112, 112, 14  0          ['block2a_expand_bn[0][0]']      
 ivation)                       4)                                                                
                                                                                                  
 block2a_dwconv_pad (ZeroPaddin  (None, 113, 113, 14  0          ['block2a_expand_activation[0][0]
 g2D)     

 ivation)                                                                                         
                                                                                                  
 block2c_dwconv (DepthwiseConv2  (None, 56, 56, 192)  1728       ['block2c_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block2c_bn (BatchNormalization  (None, 56, 56, 192)  768        ['block2c_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block2c_activation (Activation  (None, 56, 56, 192)  0          ['block2c_bn[0][0]']             
 )                                                                                                
          

 block3a_dwconv (DepthwiseConv2  (None, 28, 28, 192)  4800       ['block3a_dwconv_pad[0][0]']     
 D)                                                                                               
                                                                                                  
 block3a_bn (BatchNormalization  (None, 28, 28, 192)  768        ['block3a_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block3a_activation (Activation  (None, 28, 28, 192)  0          ['block3a_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block3a_se_squeeze (GlobalAver  (None, 192)         0           ['block3a_activation[0][0]']     
 agePoolin

 agePooling2D)                                                                                    
                                                                                                  
 block3c_se_reshape (Reshape)   (None, 1, 1, 336)    0           ['block3c_se_squeeze[0][0]']     
                                                                                                  
 block3c_se_reduce (Conv2D)     (None, 1, 1, 14)     4718        ['block3c_se_reshape[0][0]']     
                                                                                                  
 block3c_se_expand (Conv2D)     (None, 1, 1, 336)    5040        ['block3c_se_reduce[0][0]']      
                                                                                                  
 block3c_se_excite (Multiply)   (None, 28, 28, 336)  0           ['block3c_activation[0][0]',     
                                                                  'block3c_se_expand[0][0]']      
          

...

                                                                                                  
 block6b_se_expand (Conv2D)     (None, 1, 1, 1632)   112608      ['block6b_se_reduce[0][0]']      
                                                                                                  
 block6b_se_excite (Multiply)   (None, 7, 7, 1632)   0           ['block6b_activation[0][0]',     
                                                                  'block6b_se_expand[0][0]']      
                                                                                                  
 block6b_project_conv (Conv2D)  (None, 7, 7, 272)    443904      ['block6b_se_excite[0][0]']      
                                                                                                  
 block6b_project_bn (BatchNorma  (None, 7, 7, 272)   1088        ['block6b_project_conv[0][0]']   
 lization)                                                                                        
          

                                                                                                  
 block6d_project_conv (Conv2D)  (None, 7, 7, 272)    443904      ['block6d_se_excite[0][0]']      
                                                                                                  
 block6d_project_bn (BatchNorma  (None, 7, 7, 272)   1088        ['block6d_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block6d_drop (Dropout)         (None, 7, 7, 272)    0           ['block6d_project_bn[0][0]']     
                                                                                                  
 block6d_add (Add)              (None, 7, 7, 272)    0           ['block6d_drop[0][0]',           
                                                                  'block6c_add[0][0]']            
          

                                                                                                  
 block6f_drop (Dropout)         (None, 7, 7, 272)    0           ['block6f_project_bn[0][0]']     
                                                                                                  
 block6f_add (Add)              (None, 7, 7, 272)    0           ['block6f_drop[0][0]',           
                                                                  'block6e_add[0][0]']            
                                                                                                  
 block6g_expand_conv (Conv2D)   (None, 7, 7, 1632)   443904      ['block6f_add[0][0]']            
                                                                                                  
 block6g_expand_bn (BatchNormal  (None, 7, 7, 1632)  6528        ['block6g_expand_conv[0][0]']    
 ization)                                                                                         
          

                                                                                                  
 block7a_expand_conv (Conv2D)   (None, 7, 7, 1632)   443904      ['block6h_add[0][0]']            
                                                                                                  
 block7a_expand_bn (BatchNormal  (None, 7, 7, 1632)  6528        ['block7a_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block7a_expand_activation (Act  (None, 7, 7, 1632)  0           ['block7a_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block7a_dwconv (DepthwiseConv2  (None, 7, 7, 1632)  14688       ['block7a_expand_activation[0][0]
 D)       

 dense (Dense)                  (None, 7)            12551       ['global_average_pooling2d[0][0]'
                                                                 ]                                
                                                                                                  
Total params: 17,686,374
Trainable params: 17,561,167
Non-trainable params: 125,207
__________________________________________________________________________________________________


In [13]:
for layer in base_model.layers:
    layer.trainable = True

In [14]:
model.save('1013_unfreeze.h5')

In [15]:
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])

In [16]:
model.save('1013_compiled.h5')

In [17]:
es = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)

In [ ]:
model.fit(trainset, validation_data=validset, epochs=10, callbacks=[es])

In [ ]:
## already trained. screw you Markdown

In [20]:
model.evaluate(trainset)

21367/21367 [==============================] - 3075s 144ms/step - loss: 0.8490 - accuracy: 0.1871


[0.8489982485771179, 0.18708865344524384]

In [21]:
model.evaluate(validset)

1702/1702 [==============================] - 246s 144ms/step - loss: 0.5911 - accuracy: 0.1485


[0.5910961627960205, 0.14845629036426544]

In [22]:
model.save('1014_worstmodel.h5')